We set a rating (with values from $1$ to $5$) matrix with 4 movies given by 6 users. Some users didn’t watch some movies before, so the rating is given as 0 in the rating.

In [ ]:
import numpy

def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002):
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    
    Q = Q.T
    laste = 0.0
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # update parameters
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k])

        eR = numpy.dot(P,Q)

        e = 0
        
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        if e < 100:
            break

    return P, Q.T

In [ ]:
R = [
     [5,3,0,1],
     [4,0,0,1],
     [1,1,0,5],
     [1,0,0,4],
     [0,1,5,4],
     [2,1,3,0],
    ]

R = numpy.array(R)
# N: num of User
N = len(R)
# M: num of Movie
M = len(R[0])
# Num of Features
K = 3
 
P = numpy.random.rand(N,K)
Q = numpy.random.rand(M,K)

nP, nQ = matrix_factorization(R, P, Q, K)

nR = numpy.dot(nP, nQ.T)

The predicted matrix has similar output with the true values, and the 0 ratings are replaced with the prediction based on the similar users’ preferences on movies.

In [ ]:
R

array([[5, 3, 0, 1],
       [4, 0, 0, 1],
       [1, 1, 0, 5],
       [1, 0, 0, 4],
       [0, 1, 5, 4],
       [2, 1, 3, 0]])

In [ ]:
nR

array([[4.99129986, 2.99029123, 5.80431828, 1.00257357],
       [3.98379767, 1.22320988, 4.24503112, 1.00490025],
       [0.99235107, 0.98483695, 2.7686549 , 5.00416005],
       [0.98388458, 0.81268152, 2.38989039, 4.00356482],
       [3.92548391, 1.00757738, 5.01978764, 3.9885138 ],
       [2.08893367, 1.02190549, 2.91964903, 2.32110281]])

In the real-world, the rating matrix is very sparse since every user watches movies at different frequencies. However, the error function RMSE is only calculated with the non-null rating. The missing entries in the rating matrix would be replaced by the dot product of the factor matrices. Therefore, we know what to recommend to the users with the unseen movies based on the prediction.